In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

def preprocess_image(image, label):
    image = tf.image.grayscale_to_rgb(image)
    image = image / 255.0
    return image, label

train_ds = image_dataset_from_directory(
    'dataset/train',
    image_size=(96 ,96),
    validation_split=0.2,
    seed=123,
    color_mode='grayscale',
    batch_size=32,
    subset='training'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
     'dataset/train',
    image_size=(96 ,96),
    validation_split=0.2,
    seed=123,
    color_mode='grayscale',
    batch_size=32,
    subset='validation'
)

train_ds = train_ds.map(lambda x, y: preprocess_image(x, y))
val_ds = val_ds.map(lambda x, y: preprocess_image(x, y))

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(250).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


Found 28709 files belonging to 7 classes.
Using 22968 files for training.
Found 28709 files belonging to 7 classes.
Using 5741 files for validation.


In [8]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model


model = MobileNetV2(input_shape=(96, 96, 3), include_top=False, pooling='none', weights='imagenet')

layer_count = 0
for layer in model.layers:
    layer.trainable = False
    layer_count += 1

x = model.output
x = GlobalAveragePooling2D()(x)
x = Dense(640, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(7, activation='softmax')(x)

model = Model(inputs=model.input, outputs=x)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 Conv1 (Conv2D)                 (None, 48, 48, 32)   864         ['input_5[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 48, 48, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 48, 48, 32)   0           ['bn_Conv1[0][0]']               
                                                                                            

In [9]:
model.fit(
    train_ds,
    epochs=50,
    batch_size=32,
    validation_data=val_ds,
)

Epoch 1/50
718/718 [==============================] - 84s 78ms/step - loss: 1.5674 - accuracy: 0.4030 - val_loss: 1.4692 - val_accuracy: 0.4447
Epoch 2/50
718/718 [==============================] - 25s 35ms/step - loss: 1.4036 - accuracy: 0.4629 - val_loss: 1.4262 - val_accuracy: 0.4583
Epoch 3/50
718/718 [==============================] - 24s 33ms/step - loss: 1.3443 - accuracy: 0.4871 - val_loss: 1.3837 - val_accuracy: 0.4811
Epoch 4/50
718/718 [==============================] - 24s 33ms/step - loss: 1.2985 - accuracy: 0.5047 - val_loss: 1.3897 - val_accuracy: 0.4740
Epoch 5/50
718/718 [==============================] - 24s 33ms/step - loss: 1.2492 - accuracy: 0.5238 - val_loss: 1.3774 - val_accuracy: 0.4848
Epoch 6/50
718/718 [==============================] - 24s 33ms/step - loss: 1.2060 - accuracy: 0.5424 - val_loss: 1.3711 - val_accuracy: 0.4868
Epoch 7/50
718/718 [==============================] - 24s 33ms/step - loss: 1.1598 - accuracy: 0.5637 - val_loss: 1.3939 - val_accuracy:

In [12]:
i = 77
count = 0
for layer in model.layers:
    if i == count:
        layer.trainable = True
        i += 1
    count += 1

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(
    train_ds,
    epochs=50,
    batch_size=32,
    validation_data=val_ds,
)

Epoch 1/50
 80/718 [==>...........................] - ETA: 52s - loss: 0.3306 - accuracy: 0.8855

KeyboardInterrupt: 

In [5]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
     'dataset/test',
    image_size=(48 ,48),
    color_mode='grayscale',
    batch_size=32,
)

test_ds = test_ds.map(lambda x, y: preprocess_image(x, y))

AUTOTUNE = tf.data.AUTOTUNE
test_ds = test_ds.cache().shuffle(250).prefetch(buffer_size=AUTOTUNE)

test_loss, test_accuracy = model.evaluate(test_ds)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

Found 7178 files belonging to 7 classes.
225/225 [==============================] - 12s 22ms/step - loss: 1.9800 - accuracy: 0.5150
Test Loss: 1.9800227880477905
Test Accuracy: 0.5150460004806519


In [7]:
model.save('mobilenetv2_fer.h5')